## Notes

### Highway entry/exit
- Allgemein: <br>
    - Wenn anderes Auto neben ego ist, dann wird es eigetnlich nur schräg vor/hinter ego angezeigt und nicht parallel
    - Code refactoring
    - doc strings updaten
    - Kommentare anpassen
- dual exit/entry <br>
    - degree and col_distances anpassen, ggf. 2 unterschiedliche shapes für duals berücksichtigen
    - und runtern fahren / rauf fahren 
- single exit/entry und exit&entry
    - degree and col_distances anpassen
- auf exit abfahren / entry rauffahren:
    - mit exit noch mal checken

- Problem Kreuzungen:
    - Tankestelle und drum herum
    - double entry mit Ampel 

- Server
    - aufsetzen




## sonstiges
- triangles 
- check road_change
- calculate position in matrix
- grouping wps (Lukas) <br>
ego auf rechter Spur bei Einfahrt/Ausfahrt: Problem, wenn nur 3 straighte lines in remove_wps(), da in wps_on_ego_lane nur die einzelne Abfahrt drauf ist .. erkennt dann shape falsch durch falsche Gruppierung

In [4]:
[1,2,3] contains 1

SyntaxError: invalid syntax (1442333121.py, line 1)

In [1]:
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random


warnings.filterwarnings("ignore")
from utils import *

pygame 2.3.0 (SDL 2.24.2, Python 3.9.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
distance_to_junc = 10
direction_angle = 20

actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())



[Actor(id=24, type=spectator), Actor(id=25, type=traffic.stop), Actor(id=26, type=traffic.stop), Actor(id=27, type=traffic.stop), Actor(id=28, type=traffic.stop), Actor(id=29, type=traffic.stop), Actor(id=30, type=traffic.stop), Actor(id=31, type=traffic.stop), Actor(id=32, type=traffic.stop), Actor(id=33, type=traffic.unknown), Actor(id=34, type=traffic.unknown), Actor(id=35, type=traffic.yield), Actor(id=36, type=traffic.yield), Actor(id=37, type=traffic.yield), Actor(id=38, type=traffic.unknown), Actor(id=39, type=traffic.unknown), Actor(id=40, type=traffic.yield), Actor(id=41, type=traffic.speed_limit.40), Actor(id=42, type=traffic.speed_limit.40), Actor(id=43, type=traffic.speed_limit.90), Actor(id=44, type=traffic.speed_limit.90), Actor(id=45, type=traffic.speed_limit.90), Actor(id=46, type=traffic.speed_limit.90), Actor(id=47, type=traffic.speed_limit.90), Actor(id=48, type=traffic.speed_limit.90), Actor(id=49, type=traffic.speed_limit.90), Actor(id=50, type=traffic.speed_limit.

In [15]:
manual_car = world.get_actor(153)
ego_vehicle = world.get_actor(153)
second_vehicle = world.get_actor(161)
print(manual_car)
print(second_vehicle)

Actor(id=153, type=vehicle.mercedes.sprinter)
Actor(id=161, type=sensor.other.lane_invasion)


In [25]:
# def get_highway_shape(wps_grouped, junction):
#     # Check if entry or exit
#     wps_grouped.sort(key=len)
#     if len(wps_grouped[0]) == 2:
#         if wps_grouped[0][0][1] != wps_grouped[0][1][1]:
#             highway_type = "entry_and_exit"
#         elif wps_grouped[0][0][1] == "start":
#             highway_type = "dual_entry"
#         else:
#             highway_type = "dual_exit"

#     elif len(wps_grouped[0]) == 1:
#         if wps_grouped[0][0][1] == "start":
#             highway_type = "single_entry"
#         else:
#             highway_type = "single_exit"
#     else:
#         print("ERROR in Higway Shape")
#         return None

#     # Check number of lanes
#     road_ids = [wp[0].road_id for wp in wps_grouped[1]]
#     if len(set(road_ids)) == 1:
#         straight_lanes = 4
#     else:
#         straight_lanes = 3

#     entry_wps = None
#     exit_wps = None
#     # get start & end waypoint of entry / exit as tuple
#     if highway_type in ["entry_and_exit", "single_entry", "dual_entry"]:
#         entry_wps = get_exit_or_entry(wps_grouped, "entry", junction)
#     if highway_type in ["entry_and_exit", "single_exit", "dual_exit"]:
#         exit_wps = get_exit_or_entry(wps_grouped, "exit", junction)

#     return (highway_type, straight_lanes, entry_wps, exit_wps)


##########################################################################################
# def get_exit_or_entry(wps, exit_or_entry, junction):
#     junction_wps = junction.get_waypoints(carla.LaneType().Driving)

#     if exit_or_entry == "exit":
#         order = ["end", "start"]
#     elif exit_or_entry == "entry":
#         order = ["start", "end"]
#     first = []
#     second = []
#     for group in wps:
#         if len(group) != 4:
#             for wp in group:
#                 if wp[1] == order[0]:
#                     first.append(wp[0])


#     for first_wp in first:
#         for wp_pair in junction_wps:
#             if wp_pair[0].id == first_wp.id:
#                 second.append(wp_pair[1])
#                 break
#             elif wp_pair[1].id == first_wp.id:
#                 second.append(wp_pair[0])
#                 break

#     if not first or not second:
#         print("Error in get_exit()")
#         return None
#     if exit_or_entry == "exit":
#         return (second, first)  # first = end_wp, second = start_wp
#     elif exit_or_entry == "entry":
#         return (first, second)  # first = start_wp, second = end_wp
##########################################################################################
# def search_entry_or_exit(world_map, ego_vehicle, direction, exit_entry_found, matrix, highway_shape, wps, entry_wps, exit_wps, degree, junction_id):
#     ego_vehicle_location = ego_vehicle.get_location()
#     ego_waypoint = world_map.get_waypoint(ego_vehicle_location)  
#     # for dual entry/exit we compare yaw values based on most right highway lane
#     if highway_shape[0] in ["dual_exit", "dual_entry"]:
#         initial_wp = ego_waypoint
#         if direction == "before":
#             while True:
#                 tmp_right_lane = initial_wp.get_right_lane()
#                 if (tmp_right_lane is None) or (tmp_right_lane.lane_type != carla.LaneType().Driving):
#                     break
#                 initial_wp = tmp_right_lane
#         # for looking behind, we consider left line first bc. otherwise wp.previous() go on entry
#         elif direction == "behind":
#             while True:
#                 tmp_left_lane = initial_wp.get_left_lane()
#                 if (tmp_left_lane is None) or (tmp_left_lane.lane_type != carla.LaneType().Driving):
#                     break
#                 initial_wp = tmp_left_lane
#         for wps_group in wps:
#             if (len(wps_group) == 4) and (wps_group[0][1] == "end"):
#                 right_lane_end_wp = wps_group[0][0] # first wp of group
#                 for wp in wps_group:
#                     if abs(wp[0].lane_id) > abs(right_lane_end_wp.lane_id):
#                         right_lane_end_wp = wp[0]
#     # for other entry/exit we compare yaw on entry/exit
#     else:
#         initial_wp = ego_waypoint

#     col_entryExit = None
#     # Look in front
#     if direction == "before":
#         z = 3 # column help in matrix
#         col_distances = [6, 15, 35, 55, 75]
#     # Look behind
#     elif direction == "behind":
#         z = -1 # column help in matrix; -1 because in list is first value only a threshold to see when junction too far behind
#         if highway_shape[0] in ["entry_and_exit"]: # special because entry & exit are very close compared to normal column distance in matrix 
#             col_distances = [55, 35, 10, 6]
#         else:
#             col_distances = [55, 35, 15, 6]

#     for i, col_distance in enumerate(col_distances): # index 0 equals distance closest (10)
#         if exit_entry_found:
#             break
#         for j in [-5, 0, 5]: #  check 3 values in each column, bc. otherwise sometimes jumps are bigger than junction
#             # Look in front
#             if direction == "before":
#                 # if (col_distance == 4) and (j == 5):
#                 #     j = 0
#                 wp = initial_wp.next(col_distance+j)[0]
#             # Look behind
#             elif direction == "behind":
#                 #j = 0
#                 if highway_shape[0] == ["dual_entry", "dual_exit"]:
#                     wp = initial_wp.previous(col_distance+j)[0]
#                     while True:
#                         tmp_right_lane = wp.get_right_lane()
#                         if (tmp_right_lane is None) or (tmp_right_lane.lane_type != carla.LaneType().Driving):
#                             break
#                         wp = tmp_right_lane
#                 else:
#                     wp = initial_wp.previous(col_distance+j)[0]
    
#             if (wp.is_junction) and (wp.get_junction().id == junction_id):
#                 # now we have to check if parallel or curve away, depending on exit / entry as first search
#                 if highway_shape[0] in ["entry_and_exit", "single_exit"]: #search for exit first
#                     distance_junction_start = get_distance_junction_start(wp)
#                     yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(exit_wps[0][0].next(distance_junction_start)[0].transform.rotation.yaw) 
#                     already_in_curve = (abs(yaw_difference) > degree)
#                 elif highway_shape[0] in ["single_entry"]:
#                     distance_junction_end = get_distance_junction_end(wp)
#                     yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(entry_wps[1][0].previous(distance_junction_end)[0].transform.rotation.yaw) 
#                     already_in_curve = (abs(yaw_difference) < degree)
#                 elif highway_shape[0] in ["dual_entry"]:
#                     yaw_difference = abs(right_lane_end_wp.transform.rotation.yaw) - abs(wp.transform.rotation.yaw) 
#                     already_in_curve = (abs(yaw_difference) < degree)
#                 elif highway_shape[0] in ["dual_exit"]:
#                     print("ERROR: DUAL EXIT ")
                
#                 # if ego already too far away, dont show in matrix
#                 if (direction == "behind") and already_in_curve and (col_distance == 55):
#                     exit_entry_found = True 
#                     break 
#                 # if wp already in curve, update matrix
#                 elif already_in_curve: 
#                     matrix[6][1][z+i] = 0
#                     matrix[7][1][z+i] = 0
#                     col_entryExit = z+i
#                     if z+i+1 < 8: # case: curve is 70m infront (last column in matrix)
#                         if highway_shape[0] in ["entry_and_exit"]:
#                             if highway_shape[1] == 3:
#                                 matrix[5][1][z+i+1] = 3
#                             # add entry, if closer than 70m 
#                             if z+i+2 < 8:
#                                 matrix[6][1][z+i+2] = 0
#                                 matrix[7][1][z+i+2] = 0
#                         elif highway_shape[0] in ["dual_entry"]:
#                             matrix[6][1][z+i+1] = 0
#                             matrix[7][1][z+i+1] = 0
#                         elif highway_shape[0] in ["dual_exit"]:
#                             print("ERROR: DUAL EXIT")    
#                     exit_entry_found = True 
#                     break 

#     return exit_entry_found, col_entryExit, matrix

##########################################################################################
# def update_matrix(world_map, ego_vehicle, ego_location, highway_shape, wps, matrix, ego_waypoint, junction, cars_on_entryExit, ghost=False):
#     junction_id = junction.id
#     if highway_shape[1] == 3:
#         matrix[list(matrix.keys())[5]] = [0,0,0,0,0,0,0,0]
#         # TODO wenn 4 lane da, dann doppelt other cars detection? ggf. einfach immer rechte Spur auf 0en setzen?

#     matrix = list(matrix.items())
#     exit_entry_found = False 
#     entry_wps = highway_shape[2]
#     exit_wps = highway_shape[3]

#     # set the angle threshold depending on curve type
#     if highway_shape[0] == "entry_and_exit":
#         degree = 30
#     elif highway_shape[0] in ["single_entry", "single_exit"]:
#         degree = 20
#     elif highway_shape[0] in ["dual_entry", "dual_exit"]:
#         degree = 15 # TODO

   
#     # First: Look behind ego
#     exit_entry_found, col_entryExit, matrix = search_entry_or_exit(world_map, ego_vehicle, "behind", exit_entry_found, matrix, highway_shape, wps, entry_wps, exit_wps, degree, junction_id)
#     # if not yet found, look in front
#     if exit_entry_found == False:
#         _, col_entryExit, matrix = search_entry_or_exit(world_map, ego_vehicle, "before", exit_entry_found, matrix, highway_shape, wps, entry_wps, exit_wps, degree, junction_id)
    
#     # get road ids of: entry/exit, city road before and highway road after junction object
#     entry_road_id = []
#     exit_road_id = []
#     entry_city_road = []
#     exit_city_road = []
#     entry_highway_road = []
#     exit_highway_road = []
#     entry_highway_lane_id = []
#     exit_highway_lane_id = []
#     if entry_wps:
#         for entry_wp in entry_wps[0]:
#             entry_city_road.append(entry_wp.previous(3)[0].road_id)
#             entry_road_id.append(entry_wp.road_id)
#         for entry_wp in entry_wps[1]:
#             entry_highway_road.append(entry_wp.next(3)[0].road_id)
#             entry_highway_lane_id.append(abs(entry_wp.next(3)[0].lane_id))

#     if exit_wps:
#         for exit_wp in exit_wps[1]:
#             exit_city_road.append(exit_wp.next(3)[0].road_id)
#             exit_road_id.append(exit_wp.road_id)
#         for exit_wp in exit_wps[0]:
#             exit_highway_road.append(exit_wp.previous(3)[0].road_id)
#             exit_highway_lane_id.append(abs(exit_wp.previous(3)[0].lane_id))

#     # get right_lane_end_wp for dual entry suroounding cars angle calculation
#     for wps_group in wps:
#         if (len(wps_group) == 4) and (wps_group[0][1] == "end"):
#             right_lane_end_wp = wps_group[0][0] # first wp of group
#             for wp in wps_group:
#                 if abs(wp[0].lane_id) > abs(right_lane_end_wp.lane_id):
#                     right_lane_end_wp = wp[0]


#     # detect surrounding cars on exit/entry
#     for car in cars_on_entryExit:
#         col = calculate_position_in_matrix(ego_vehicle, car, matrix, world_map)
#         if matrix:
#             other_car_waypoint = world_map.get_waypoint(car.get_location())
#             other_car_road_id = other_car_waypoint.road_id
#             if other_car_road_id in entry_city_road+exit_city_road:
#                 # ignore cars that are before junction object on city road
#                 continue
#             elif other_car_road_id in entry_road_id+entry_highway_road:
#                 if highway_shape[0] == "dual_entry":
#                     distance_junction_end = get_distance_junction_end(other_car_waypoint)
#                     yaw_difference = abs(right_lane_end_wp.transform.rotation.yaw) - abs(right_lane_end_wp.previous(distance_junction_end)[0].transform.rotation.yaw)
#                 else:
#                     distance_junction_end = get_distance_junction_end(other_car_waypoint)
#                     yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(entry_wps[1][0].previous(distance_junction_end)[0].transform.rotation.yaw) 
#             elif other_car_road_id in exit_road_id+exit_highway_road:
#                 distance_junction_start = get_distance_junction_start(other_car_waypoint)
#                 yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(exit_wps[0][0].next(distance_junction_start)[0].transform.rotation.yaw) 

#             already_in_curve = (abs(yaw_difference) > degree)
#             already_after_curve = (abs(yaw_difference) > degree*2)
#             if already_after_curve:
#                 row = 7
#             elif already_in_curve:
#                 row = 6
#             else:
#                 row = 5

#             if already_in_curve:
#                 if highway_shape[0] in ["entry_and_exit"]:
#                     col_entry = col_entryExit + 2
#                     col_exit = col_entryExit
#                 elif highway_shape[0] in ["dual_entry", "dual_exit"]:
#                     # dual entry
#                     entry_road_id = [x for _,x in sorted(zip(entry_highway_lane_id,entry_road_id))]
#                     entry_highway_lane_id.sort()
#                     if other_car_road_id == entry_road_id[0]:
#                         col_entry = col_entryExit
#                     elif other_car_road_id == entry_road_id[1]:
#                         col_entry = col_entryExit + 1

#                     # dual exit
#                     exit_road_id = [x for _,x in sorted(zip(exit_highway_lane_id,exit_road_id))]
#                     exit_highway_lane_id.sort()
#                     if other_car_road_id == exit_road_id[0]:
#                         col_exit = col_entryExit
#                     elif other_car_road_id == exit_road_id[1]:
#                         col_exit = col_entryExit + 1

#                 else:
#                     col_entry = col_entryExit
#                     col_exit = col_entryExit

#             else:
#                 if (highway_shape[1] == 3) and (col <= (col_entryExit + 1)):
#                     col_entry = col_entryExit + 2
#                 else:
#                     col_entry = col
#                 col_exit = col

#             if (other_car_road_id in entry_road_id+entry_highway_road) and (col_entry < 8):
#                 matrix[row][1][col_entry] = 2
#             elif (other_car_road_id in exit_road_id+exit_highway_road) and (col_exit < 8):
#                 matrix[row][1][col_exit] = 2


#             # TODO: check ob gerade auf Ego

#     print(col)
#     print(col_entry)
#     print(col_exit)
#     return dict(matrix)
##########################################################################################
# def detect_surronding_cars(ego_vehicle, matrix, road_lane_ids, world, radius, on_highway, highway_shape):
#     """
#     Detect surrounding cars and update the city matrix accordingly.

#     Parameters:
#         ego_vehicle (Vehicle): The ego vehicle for which to detect surrounding cars.
#         matrix (collections.OrderedDict): The city matrix representing the lanes around the ego vehicle.
#         road_lanes_ids (list of str): List of road and lane IDs in the world map.
#         world (carla.World): The game world where the simulation is running.
#         radius (float): The radius within which to detect surrounding cars.

#     Returns:
#         collections.OrderedDict: The updated city matrix with detected surrounding cars.

#     Description:
#         This function detects surrounding cars within a specified radius from the ego vehicle and
#         updates the city matrix accordingly. The city matrix is represented as an ordered dictionary
#         with lane IDs as keys and lists of integers as values. Each list corresponds to the direction
#         of the lane, where:
#             - 0 indicates a lane in the same direction as the ego vehicle's current lane.
#             - 1 indicates the ego vehicle's current lane.
#             - 2 indicates a lane with a detected surrounding car.
#             - 3 indicates an empty lane with no traffic.

#         The function calculates the position of the detected cars in the city matrix and updates the
#         corresponding cells in the matrix to indicate the presence of surrounding cars. The matrix is
#         updated based on the lanes and positions/distance of the detected cars relative to the ego vehicle.

#     Note:
#         The city matrix should be pre-generated using the 'create_city_matrix' function.
#         The returned city matrix may contain lane directions that are not within the ego vehicle's vicinity,
#         as determined by the input road_lane_ids and world.
#     """
#     world_map = world.get_map()
#     # Get all surronding cars in specified radius
#     surrounding_cars = []
#     surrounding_cars_on_highway_entryExit = []
#     ego_vehicle_waypoint = world_map.get_waypoint(ego_vehicle.get_location())
#     ego_vehicle_road_id = ego_vehicle_waypoint.road_id

#     for actor in world.get_actors():
#         if "vehicle" in actor.type_id and actor.id != ego_vehicle.id:
#             distance_to_actor = distance(
#                 actor.get_location(), ego_vehicle.get_location()
#             )
#             if distance_to_actor <= radius:
#                 surrounding_cars.append(actor)

#     _, next_lanes = check_road_change(ego_vehicle, road_lane_ids, True, world_map)
#     _, prev_lanes = check_road_change(ego_vehicle, road_lane_ids, False, world_map)
#     lanes_exist_futher = False
#     lanes = []
#     for id in road_lane_ids:
#         if str(ego_vehicle_road_id) == id.split("_")[0]:
#             lanes.append(id.split("_")[1])
#     try:
#         if next_lanes and matrix:
#             # lanes = [road_lane.split("_")[1] for road_lane in matrix.keys()]
#             lanes_exist_futher = all(lane in next_lanes for lane in lanes) or all(
#                 lane in lanes for lane in next_lanes
#             )
#     except IndexError:
#         pass

#     lanes_existed_before = False
#     try:
#         if prev_lanes and matrix:
#             # lanes = [road_lane.split("_")[1] for road_lane in matrix.keys()]
#             lanes_existed_before = all(lane in prev_lanes for lane in lanes) or all(
#                 lane in lanes for lane in prev_lanes
#             )
#     except IndexError:
#         pass
    
#     # in the following, ignore cars that are on highway exit/entry lanes 
#     entry_wps = highway_shape[2]
#     exit_wps = highway_shape[3]
#     if not highway_shape is None:
#         entry_road_id = []
#         exit_road_id = []
#         entry_city_road = []
#         exit_city_road = []
#         entry_highway_road = []
#         exit_highway_road = []
#         if entry_wps:
#             for entry_wp in entry_wps[0]:
#                 entry_city_road.append(entry_wp.previous(3)[0].road_id)
#                 entry_road_id.append(entry_wp.road_id)
#             for entry_wp in entry_wps[1]:
#                 entry_highway_road.append(entry_wp.next(3)[0].road_id)
#         if exit_wps:
#             for exit_wp in exit_wps[1]:
#                 exit_city_road.append(exit_wp.next(3)[0].road_id)
#                 exit_road_id.append(exit_wp.road_id)
#             for exit_wp in exit_wps[0]:
#                 exit_highway_road.append(exit_wp.previous(3)[0].road_id)

#         # if highway_shape[0] in ["entry_and_exit", "single_entry", "dual_entry"]:
#         #     entry_road_id.append(highway_shape[2][0][0].road_id)
#         #     entry_city_road.append(highway_shape[2][0][0].previous(3)[0].road_id)
#         #     entry_highway_road.append()
#         # if highway_shape[0] == "dual_entry":
#         #     entry_road_id.append(highway_shape[2][0][1].road_id)
#         #     entry_city_road.append(highway_shape[2][0][1].previous(3)[0].road_id)
#         # if highway_shape[0] in ["entry_and_exit", "single_exit", "dual_exit"]:
#         #     exit_road_id.append(highway_shape[3][1][0].road_id)
#         #     exit_city_road.append(highway_shape[3][1][0].next(3)[0].road_id)
#         # if highway_shape[0] == "dual_exit":
#         #     exit_road_id.append(highway_shape[3][1][1].road_id)
#         #     exit_city_road.append(highway_shape[3][1][1].next(3)[0].road_id)



#     # Update matrix based on the lane and position/distance to ego vehicle of other car
#     for car in surrounding_cars:
#         # Get road and lane_id of other car
#         other_car_waypoint = world_map.get_waypoint(car.get_location())
#         other_car_lane_id = other_car_waypoint.lane_id
#         other_car_road_id = other_car_waypoint.road_id
#         other_car_road_lane_id = str(other_car_road_id) + "_" + str(other_car_lane_id)

#         # ignore car on highway entry / Exit
#         if on_highway and (not highway_shape is None) and (other_car_road_id in entry_road_id+entry_city_road+exit_road_id+exit_city_road+exit_highway_road+entry_highway_road):
#             surrounding_cars_on_highway_entryExit.append(car)
#             continue

#         col = calculate_position_in_matrix(ego_vehicle, car, matrix, world_map)
#         if col is None:
#             continue

#         if matrix:
#             if other_car_road_lane_id in matrix.keys():
#                 matrix[other_car_road_lane_id][col] = 2
#                 continue
#             elif (lanes_exist_futher or lanes_existed_before) and str(
#                 other_car_lane_id
#             ) in [str(road_lane.split("_")[1]) for road_lane in matrix.keys()]:
#                 matrix[str(ego_vehicle_road_id) + "_" + str(other_car_lane_id)][col] = 2
#     print(surrounding_cars_on_highway_entryExit)
#     return matrix, surrounding_cars_on_highway_entryExit




In [67]:
# def update_matrix(
#     world_map,
#     ego_vehicle,
#     ego_location,
#     highway_shape,
#     wps,
#     matrix,
#     ego_waypoint,
#     junction,
#     cars_on_entryExit,
#     ghost=False,
# ):
#     junction_id = junction.id
#     if highway_shape[1] == 3:
#         matrix[list(matrix.keys())[5]] = [0, 0, 0, 0, 0, 0, 0, 0]
#         # TODO wenn 4 lane da, dann doppelt other cars detection? ggf. einfach immer rechte Spur auf 0en setzen?

#     matrix = list(matrix.items())
#     exit_entry_found = False
#     entry_wps = highway_shape[2]
#     exit_wps = highway_shape[3]

#     # set the angle threshold depending on curve type
#     if highway_shape[0] == "entry_and_exit":
#         degree = 30
#     elif highway_shape[0] in ["single_entry", "single_exit"]:
#         degree = 20
#     elif highway_shape[0] in ["dual_entry"]:
#         degree = 18  # TODO: test and fine tuning of degree
#     elif highway_shape[0] in ["dual_exit"]:
#         degree = 18  # TODO: test and fine tuning of degree

#     # get right_lane_end_wp & right_lane_start_wp for dual entry & exit surrounding cars angle calculation
#     for wps_group in wps:
#         if (len(wps_group) == 4) and (wps_group[0][1] == "end"):
#             right_lane_end_wp = wps_group[0][0] # first wp of group
#             for wp in wps_group:
#                 if abs(wp[0].lane_id) > abs(right_lane_end_wp.lane_id):
#                     right_lane_end_wp = wp[0]
#         elif (len(wps_group) == 4) and (wps_group[0][1] == "start"):
#             right_lane_start_wp = wps_group[0][0] # first wp of group
#             for wp in wps_group:
#                 if abs(wp[0].lane_id) > abs(right_lane_start_wp.lane_id):
#                     right_lane_start_wp = wp[0]

#     # First: Look behind ego
#     exit_entry_found, col_entryExit, matrix = search_entry_or_exit(
#         world_map,
#         ego_location,
#         "behind",
#         exit_entry_found,
#         matrix,
#         highway_shape,
#         wps,
#         entry_wps,
#         exit_wps,
#         degree,
#         junction_id,
#         right_lane_end_wp, # TODO: neuer param
#         right_lane_start_wp, # TODO: neuer param
#     )
#     # if not yet found, look in front
#     if exit_entry_found == False:
#         _, col_entryExit, matrix = search_entry_or_exit(
#             world_map,
#             ego_location,
#             "before",
#             exit_entry_found,
#             matrix,
#             highway_shape,
#             wps,
#             entry_wps,
#             exit_wps,
#             degree,
#             junction_id,
#             right_lane_end_wp, # TODO: neuer param
#             right_lane_start_wp, # TODO: neuer param
#         )

#     # get road ids of: entry/exit, city road before and highway road after junction object
#     entry_road_id = []
#     exit_road_id = []
#     entry_city_road = []
#     exit_city_road = []
#     entry_highway_road = []
#     exit_highway_road = []
#     entry_highway_lane_id = []
#     exit_highway_lane_id = []
#     if entry_wps:
#         for entry_wp in entry_wps[0]:
#             entry_city_road.append(entry_wp.previous(3)[0].road_id)
#             entry_road_id.append(entry_wp.road_id)
#         for entry_wp in entry_wps[1]:
#             entry_highway_road.append(entry_wp.next(3)[0].road_id)
#             entry_highway_lane_id.append(abs(entry_wp.next(3)[0].lane_id))

#     if exit_wps:
#         for exit_wp in exit_wps[1]:
#             exit_city_road.append(exit_wp.next(3)[0].road_id)
#             exit_road_id.append(exit_wp.road_id)
#         for exit_wp in exit_wps[0]:
#             exit_highway_road.append(exit_wp.previous(3)[0].road_id)
#             exit_highway_lane_id.append(abs(exit_wp.previous(3)[0].lane_id))

#     # detect surrounding cars on exit/entry
#     for car in cars_on_entryExit:
#         col = calculate_position_in_matrix(
#             ego_location,
#             ego_vehicle,
#             car,
#             matrix,
#             world_map,
#             ego_vehicle.get_velocity(),
#             ghost,
#         )
#         if col is None:
#             continue
#         if matrix:
#             other_car_waypoint = world_map.get_waypoint(car.get_location())
#             other_car_road_id = other_car_waypoint.road_id
#             if other_car_road_id in entry_city_road + exit_city_road:
#                 # ignore cars that are before junction object on city road
#                 continue
#             elif other_car_road_id in entry_road_id + entry_highway_road:
#                 if highway_shape[0] == "dual_entry": 
#                     distance_junction_end = get_distance_junction_end(other_car_waypoint)
#                     yaw_difference = abs(right_lane_end_wp.transform.rotation.yaw) - abs(right_lane_end_wp.previous(distance_junction_end)[0].transform.rotation.yaw)
#                 else:                
#                     distance_junction_end = get_distance_junction_end(other_car_waypoint)
#                     yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(
#                         entry_wps[1][0]
#                         .previous(distance_junction_end)[0]
#                         .transform.rotation.yaw
#                     )
#             elif other_car_road_id in exit_road_id + exit_highway_road:
#                 if highway_shape[0] == "dual_exit":
#                     distance_junction_start = get_distance_junction_start(other_car_waypoint)
#                     yaw_difference = abs(right_lane_start_wp.transform.rotation.yaw) - abs(right_lane_start_wp.next(distance_junction_start)[0].transform.rotation.yaw)

#                 else:
#                     distance_junction_start = get_distance_junction_start(
#                         other_car_waypoint
#                     )
#                     yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(
#                         exit_wps[0][0]
#                         .next(distance_junction_start)[0]
#                         .transform.rotation.yaw
#                     )

#             already_in_curve = abs(yaw_difference) > degree
#             already_after_curve = abs(yaw_difference) > degree * 2
#             if already_after_curve:
#                 row = 7
#             elif already_in_curve:
#                 row = 6
#             else:
#                 row = 5

#             if already_in_curve:
#                 if highway_shape[0] in ["entry_and_exit"]:
#                     col_entry = col_entryExit + 2
#                     col_exit = col_entryExit
#                 elif highway_shape[0] in ["dual_entry"]:
#                     # dual entry
#                     entry_road_id = [x for _,x in sorted(zip(entry_highway_lane_id,entry_road_id))]
#                     entry_highway_lane_id.sort()
#                     if other_car_road_id == entry_road_id[0]: 
#                         col_entry = col_entryExit
#                     elif other_car_road_id == entry_road_id[1]:
#                         col_entry = col_entryExit + 1
#                 elif highway_shape[0] in ["dual_exit"]:
#                     # dual exit
#                     exit_road_id = [x for _,x in sorted(zip(exit_highway_lane_id,exit_road_id))]
#                     exit_highway_lane_id.sort()
#                     if other_car_road_id == exit_road_id[0]:
#                         col_exit = col_entryExit + 1
#                     elif other_car_road_id == exit_road_id[1]:
#                         col_exit = col_entryExit #+ 1
#                 else:
#                     col_entry = col_entryExit
#                     col_exit = col_entryExit
#             else:
#                 if (highway_shape[1] == 3) and (col <= (col_entryExit + 1)):
#                     col_entry = col_entryExit + 2
#                 else:
#                     if highway_shape[0] in ["dual_entry"]:
#                         # dual entry
#                         entry_road_id = [x for _,x in sorted(zip(entry_highway_lane_id,entry_road_id))]
#                         entry_highway_lane_id.sort()
#                         if other_car_road_id == entry_road_id[0]: 
#                             row = row - 1
#                     elif highway_shape[0] in ["dual_exit"]:
#                         # dual exit
#                         exit_road_id = [x for _,x in sorted(zip(exit_highway_lane_id,exit_road_id))]
#                         exit_highway_lane_id.sort()
#                         if other_car_road_id == exit_road_id[0]:
#                             row = row - 1
#                     col_entry = col
#                 col_exit = col

#             if (other_car_road_id in entry_road_id + entry_highway_road) and (
#                 col_entry < 8
#             ):
#                 if car.id == ego_vehicle.id:
#                     matrix[row][1][col_entry] = 1
#                 else:
#                     matrix[row][1][col_entry] = 2
#             elif (other_car_road_id in exit_road_id + exit_highway_road) and (col_exit < 8):
#                 if car.id == ego_vehicle.id:
#                     matrix[row][1][col_exit] = 1
#                 else:
#                     matrix[row][1][col_exit] = 2

#             # TODO: check ob gerade auf Ego
#     return dict(matrix)

In [73]:
# def search_entry_or_exit(
#     world_map,
#     ego_vehicle_location,
#     direction,
#     exit_entry_found,
#     matrix,
#     highway_shape,
#     wps,
#     entry_wps,
#     exit_wps,
#     degree,
#     junction_id,
#     right_lane_end_wp, # TODO: neuer param
#     right_lane_start_wp, # TODO: neuer param
# ):
#     ego_waypoint = world_map.get_waypoint(ego_vehicle_location)

#     # for dual entry/exit we compare yaw values based on most right highway lane
#     if highway_shape[0] in ["dual_exit", "dual_entry"]:
#         initial_wp = ego_waypoint
#         if direction == "before":
#             while True:
#                 tmp_right_lane = initial_wp.get_right_lane()
#                 if (tmp_right_lane is None) or (
#                     tmp_right_lane.lane_type != carla.LaneType().Driving
#                 ):
#                     break
#                 initial_wp = tmp_right_lane
#         # for looking behind, we consider left line first bc. otherwise wp.previous() go on entry
#         elif direction == "behind":
#             while True:
#                 tmp_left_lane = initial_wp.get_left_lane()
#                 if (tmp_left_lane is None) or (
#                     tmp_left_lane.lane_type != carla.LaneType().Driving
#                 ):
#                     break
#                 initial_wp = tmp_left_lane
# # TODO hier gelöscht 

#     # for other entry/exit we compare yaw on entry/exit
#     else:
#         initial_wp = ego_waypoint

#     col_entryExit = None
#     # Look in front
#     if direction == "before":
#         z = 3  # column help in matrix
#         col_distances = [6, 15, 35, 55, 75]
#     # Look behind
#     elif direction == "behind":
#         z = (
#             -1
#         )  # column help in matrix; -1 because in list is first value only a threshold to see when junction too far behind
#         if highway_shape[0] in [
#             "entry_and_exit"
#         ]:  # special because entry & exit are very close compared to normal column distance in matrix
#             col_distances = [55, 35, 10, 6]
#         if highway_shape[0] in [
#             "dual_exit"
#         ]:  # special because for dual exit 
#             col_distances = [55, 35, 20, 6]
#         else:
#             col_distances = [55, 35, 15, 6]
#     for i, col_distance in enumerate(
#         col_distances
#     ):  # index 0 equals distance closest (10)
#         if exit_entry_found:
#             break
#         for j in [
#             -5,
#             0,
#             5,
#         ]:  #  check 3 values in each column, bc. otherwise sometimes jumps are bigger than junction
#             # Look in front
#             if direction == "before":
#                 # if (col_distance == 4) and (j == 5):
#                 #     j = 0
#                 wp = initial_wp.next(col_distance + j)[0]
#             # Look behind
#             elif direction == "behind":
#                 # j = 0
#                 if highway_shape[0] == ["dual_entry", "dual_exit"]:
#                     wp = initial_wp.previous(col_distance + j)[0]
#                     while True:
#                         tmp_right_lane = wp.get_right_lane()
#                         if (tmp_right_lane is None) or (
#                             tmp_right_lane.lane_type != carla.LaneType().Driving
#                         ):
#                             break
#                         wp = tmp_right_lane
#                 else:
#                     wp = initial_wp.previous(col_distance + j)[0]
#             if (wp.is_junction) and (wp.get_junction().id == junction_id):
#                 # now we have to check if parallel or curve away, depending on exit / entry as first search
#                 if highway_shape[0] in [
#                     "entry_and_exit",
#                     "single_exit",
#                 ]:  # search for exit first
#                     distance_junction_start = get_distance_junction_start(wp)
#                     yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(
#                         exit_wps[0][0]
#                         .next(distance_junction_start)[0]
#                         .transform.rotation.yaw
#                     )
#                     already_in_curve = abs(yaw_difference) > degree
#                 elif highway_shape[0] in ["single_entry"]:
#                     distance_junction_end = get_distance_junction_end(wp)
#                     yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(
#                         entry_wps[1][0]
#                         .previous(distance_junction_end)[0]
#                         .transform.rotation.yaw
#                     )
#                     already_in_curve = abs(yaw_difference) < degree
#                 elif highway_shape[0] in ["dual_entry"]:
#                     yaw_difference = abs(
#                         right_lane_end_wp.transform.rotation.yaw
#                     ) - abs(wp.transform.rotation.yaw)
#                     print(direction)
#                     print(yaw_difference)
#                     print(col_distance)
#                     already_in_curve = abs(yaw_difference) < degree
#                 elif highway_shape[0] in ["dual_exit"]:
#                     yaw_difference = abs( # TODO NEU
#                         right_lane_end_wp.transform.rotation.yaw
#                     ) - abs(wp.transform.rotation.yaw)
#                     already_in_curve = abs(yaw_difference) > degree

#                 # if ego already too far away, dont show in matrix
#                 if (
#                     (direction == "behind")
#                     and already_in_curve
#                     and (col_distance == 55)
#                 ):
#                     exit_entry_found = True
#                     break

#                 # if wp already in curve, update matrix
#                 if already_in_curve:
#                     matrix[6][1][z + i] = 0
#                     matrix[7][1][z + i] = 0
#                     col_entryExit = z + i
#                     if (
#                         z + i + 1 < 8
#                     ):  # case: curve is 70m infront (last column in matrix)
#                         if highway_shape[0] in ["entry_and_exit"]:
#                             if highway_shape[1] == 3:
#                                 matrix[5][1][z + i + 1] = 3
#                             # add entry, if closer than 70m
#                             if z + i + 2 < 8:
#                                 matrix[6][1][z + i + 2] = 0
#                                 matrix[7][1][z + i + 2] = 0
#                         elif highway_shape[0] in ["dual_entry"]:
#                             matrix[6][1][z + i + 1] = 0
#                             matrix[7][1][z + i + 1] = 0
#                         elif highway_shape[0] in ["dual_exit"]:
#                             matrix[6][1][z + i + 1] = 0
#                             matrix[7][1][z + i + 1] = 0
#                     exit_entry_found = True
#                     break

#     return exit_entry_found, col_entryExit, matrix


In [74]:
# Highway Test Main
same_junction = False
on_junction = False
junction_old = None
wps_old = None
highway_shape_old = None
highway_shape = None

while True:
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    road_lane_ids = get_all_road_lane_ids(world_map = world.get_map())
    is_highway = check_ego_on_highway(ego_location, road_lane_ids, world_map)
    radius = 100

    if is_highway and ego_waypoint.is_junction:
        on_junction = True
    elif is_highway and on_junction:
        junction_old = junction
        wps_old = wps
        highway_shape_old = highway_shape
        on_junction = False
        same_junction = False

    if is_highway and (is_junction_ahead(ego_waypoint, 80) or is_junction_behind(ego_waypoint, 40)):
        if not same_junction and is_junction_ahead(ego_waypoint, 80):
            same_junction = True
            print("JUNCTION")
            junction = get_junction_ahead(ego_waypoint, 80)

            wps = junction.get_waypoints(carla.LaneType().Driving)
            closest_waypoint = get_clostest_starting_waypoint(wps, ego_location)
            wps = prepare_waypoints(closest_waypoint, wps)
            
            highway_shape = get_highway_shape(wps, junction)
            if not highway_shape is None:
                print(highway_shape[0], highway_shape[1])
        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old

        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)            
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, is_highway, highway_shape
            )
            if not highway_shape is None:
                matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape, wps, matrix, ego_waypoint, junction, cars_on_entryExit) # TODO Write this function
                if not junction_old is None:
                    matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape_old, wps_old, matrix, ego_waypoint, junction_old, cars_on_entryExit) # TODO Write this function

        for key, value in matrix.items():
            #print(key, value)
            print(value)
        print("----------------------------------------------------------------------------------------------------------------------")

    elif is_junction_ahead(ego_waypoint, 40) and not is_highway and is_highway_junction(ego_vehicle, ego_waypoint, get_junction_ahead(ego_waypoint, 40), road_lane_ids, direction_angle):
        junction = get_junction_ahead(ego_waypoint, 40)
        wps = junction.get_waypoints(carla.LaneType().Driving)
        closest_waypoint, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)

        wps = prepare_waypoints(closest_waypoint, wps)
        highway_shape = get_highway_shape(wps, junction)

        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old


        matrix = create_city_matrix(middle_location, road_lane_ids, world_map, ghost=True)
        if matrix:
            
            matrix, cars_on_entryExit = detect_surronding_cars(
                middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
            )

            if not highway_shape is None:
                matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, closest_waypoint, junction, cars_on_entryExit, True) 
                if not junction_old is None:
                    matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape_old, wps_old, matrix, closest_waypoint, junction_old, cars_on_entryExit, True) 
           
        for key, value in matrix.items():
            #print(key, value)
            print(value)
        print("----------")

    else:
        same_junction = False
        print("NO HIGHWAY JUNCTION")

    time.sleep(1)
    world.tick()

JUNCTION
dual_exit 4
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 2, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 0, 0, 3, 3]
[3, 3, 3, 3, 0, 0, 3, 3]
----------------------------------------------------------------------------------------------------------------------
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 2, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 0, 0, 3, 3]
[3, 3, 3, 3, 0, 0, 3, 3]
----------------------------------------------------------------------------------------------------------------------
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 2, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 0, 0, 3, 3]
[3, 3, 3, 3, 0, 0, 3, 3]
----------------------------------------------------------------------------------------------------------------------
[3, 3, 3, 3, 3, 3, 3, 

KeyboardInterrupt: 

# Tests

In [29]:
# TEST: get ego road/lane id loop
while True:
    time.sleep(0.3)
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    print("#####################")
    print(ego_waypoint.road_id)
    print(ego_waypoint.is_junction)

#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
#####################
1601
True
########

KeyboardInterrupt: 

In [120]:
# TEST: get ego information single moment
ego_location = ego_vehicle.get_location()
ego_waypoint = world_map.get_waypoint(ego_location)
print(ego_waypoint.is_junction)
right_lane = ego_waypoint.get_right_lane()
# print(ego_waypoint.road_id)
# print(right_lane.__str__())
# print(right_lane.road_id)
is_junction_ahead(ego_waypoint, 80)
#is_junction_behind(ego_waypoint, 40)
# print(ego_waypoint.transform.location)

False


True

In [121]:
ego_location = ego_vehicle.get_location()
ego_waypoint = world_map.get_waypoint(ego_location)
initial_wp = ego_waypoint
while True:
    tmp_right_lane = initial_wp.get_right_lane()
    if tmp_right_lane.lane_type != carla.LaneType().Driving:
        break
    initial_wp = tmp_right_lane
print(ego_waypoint.lane_id)
print(initial_wp.lane_id)

4
6


In [124]:
initial_wp.previous(19)[0].road_id

1594

In [63]:
ego_location = ego_vehicle.get_location()
ego_waypoint = world_map.get_waypoint(ego_location)
right_lane = ego_waypoint.get_right_lane()
print(ego_waypoint.lane_change)
if not right_lane is None:
    print(right_lane.road_id)
    print(right_lane.lane_id)
    print(right_lane.lane_type)

Left
1601
7
Shoulder


In [31]:
# TEST: get second vehicle information single moment
second_vehicle_location = second_vehicle.get_location()
second_vehicle_waypoint = world_map.get_waypoint(second_vehicle_location)
print(second_vehicle_waypoint.road_id)



5


In [107]:
# TEST: generate grouped wps in junction & highway shape
ego_location = ego_vehicle.get_location()
ego_waypoint = world_map.get_waypoint(ego_location)
ego_waypoint.is_junction
junction = ego_waypoint.get_junction()
junction = get_junction_ahead(ego_waypoint, 80)
wps = junction.get_waypoints(carla.LaneType().Driving)

closest_waypoint = get_clostest_starting_waypoint(wps, ego_vehicle)
print(closest_waypoint)
wps = prepare_waypoints(closest_waypoint, wps)
get_highway_shape(wps, junction)


Waypoint(Transform(Location(x=405.375793, y=-55.606678, z=0.000000), Rotation(pitch=360.000000, yaw=-89.560883, roll=0.000000)))
ERROR in Higway Shape


In [115]:
# TEST: generate grouped wps in junction & highway shape
ego_location = ego_vehicle.get_location()
ego_waypoint = world_map.get_waypoint(ego_location)
ego_waypoint.is_junction
junction = ego_waypoint.get_junction()
junction = get_junction_ahead(ego_waypoint, 80)
wps = junction.get_waypoints(carla.LaneType().Driving)

closest_waypoint = get_clostest_starting_waypoint(wps, ego_vehicle)
print(closest_waypoint)
wps = prepare_waypoints(closest_waypoint, wps)
get_highway_shape(wps, junction)


Waypoint(Transform(Location(x=377.066956, y=24.700653, z=0.021045), Rotation(pitch=359.749359, yaw=326.466492, roll=0.000000)))


('dual_entry',
 4,
 ([<carla.libcarla.Waypoint at 0x285bbbaf6d0>,
   <carla.libcarla.Waypoint at 0x285bbbaf970>]),
 None)

In [79]:
for wps_group in wps:
    if (len(wps_group) == 4) and (wps_group[0][1] == "end"):
        right_lane_end_wp = wps_group[0][0] # first wp of group
        for wp in wps_group:
            if abs(wp[0].lane_id) > abs(right_lane_end_wp.lane_id):
                right_lane_end_wp = wp[0]
        break
print(right_lane_end_wp.lane_id)

6


In [114]:
# TEST: Print grouped wps in junction
for group in wps:
    for wp in group:
        
        if wp[0]:
            print(wp[0].__str__(), wp[1])
            print(wp[0].road_id)
            print(wp[0].lane_id)
            #print("yaw:", wp[0].get_forward_vector())
    print("----------------------------------------------------------------------------------------------------------------------")

Waypoint(Transform(Location(x=363.532532, y=4.278364, z=0.021045), Rotation(pitch=0.250632, yaw=146.466476, roll=0.000000))) end
1602
-4
Waypoint(Transform(Location(x=365.466034, y=7.195834, z=0.021045), Rotation(pitch=0.250632, yaw=146.466476, roll=0.000000))) end
1602
-3
Waypoint(Transform(Location(x=367.399506, y=10.113304, z=0.021045), Rotation(pitch=0.250632, yaw=146.466476, roll=0.000000))) end
1602
-2
Waypoint(Transform(Location(x=369.333008, y=13.030774, z=0.021045), Rotation(pitch=0.250632, yaw=146.466476, roll=0.000000))) end
1602
-1
----------------------------------------------------------------------------------------------------------------------
Waypoint(Transform(Location(x=383.104889, y=-18.292522, z=0.000000), Rotation(pitch=0.000000, yaw=102.902794, roll=0.000000))) start
1602
-3
Waypoint(Transform(Location(x=379.693237, y=-19.074064, z=0.000000), Rotation(pitch=0.000000, yaw=102.902794, roll=0.000000))) start
1602
-4
Waypoint(Transform(Location(x=386.516510, y=-17.5

In [ ]:
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 3, 3, 0, 0, 0]
[3, 3, 0, 0, 0, 0, 3, 3]
[3, 3, 3, 0, 0, 3, 3, 3]

In [3]:
# Complete Main Loop

beforeJunction = False
junction_shape = "None"
matrix = None
same_junction = False
radius = 200
road_lane_ids = get_all_road_lane_ids(world_map)
while True:
    time.sleep(1)
    manual_car_location = manual_car.get_location()
    manual_car_waypoint = world_map.get_waypoint(manual_car_location)
    ego_waypoint = world_map.get_waypoint(manual_car_location)
    junction = None
    highway = False
    
    if manual_car_waypoint.next(distance_to_junc)[0].is_junction:
        junction = manual_car_waypoint.next(distance_to_junc)[0].get_junction()
        if not same_junction:
            same_junction = True
            highway = is_highway(ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle)
    
    elif manual_car_waypoint.is_junction:
        junction = manual_car_waypoint.get_junction()
       

    if manual_car_waypoint.is_junction and not highway and junction.id != 459:
        if junction_shape != "None":
            matrix = detect_surrounding_cars_outside_junction(deepcopy(junction_shape), junction_roads_junction, lanes_all_junction, ego_vehicle, world, distance_to_junc, junction)
            matrix = detect_cars_inside_junction(matrix, ego_vehicle, junction, yaw, world)
            for key, value in matrix.items():
                print(key, value)

    elif manual_car_waypoint.next(distance_to_junc)[0].is_junction and not highway and junction.id != 459:
        junction = manual_car_waypoint.next(distance_to_junc)[0].get_junction()
        junction_shape, lanes_all_junction, junction_roads_junction, yaw = getJunctionShape(ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle)
            
        
        if junction_shape != "Error":
            print("Junction ahead:")
            matrix = detect_ego_before_junction(deepcopy(junction_shape), junction_roads_junction, lanes_all_junction, ego_waypoint.lane_id, ego_waypoint, distance_to_junc)
            matrix = detect_surrounding_cars_outside_junction(matrix, junction_roads_junction, lanes_all_junction, ego_vehicle, world, distance_to_junc, junction)

            matrix= detect_cars_inside_junction(matrix, ego_vehicle, junction, yaw, world) 

            
            for key, value in dict(matrix).items():
                print(key, value)
                
                
    # else normal road:
    else:
        same_junction = False
        in_junction = False
        matrix = create_city_matrix(ego_vehicle, road_lane_ids, world_map)

        if matrix:
            matrix = detect_surronding_cars(
                ego_vehicle, matrix, road_lane_ids, world, radius
            )
            for key, value in matrix.items():
                print(key, value)
    print("======================================================================")
   
    time.sleep(0.3)
    world.tick()
    
"""       else:
same_junction = False
print("lane:", manual_car_waypoint.lane_id, "Road:", manual_car_waypoint.road_id)
# print("--------------------------------------------------------")"""

left_outer_lane [3, 3, 3, 3, 3, 3, 3, 3]
left_inner_lane [3, 3, 3, 3, 3, 3, 3, 3]
No opposing direction [3, 3, 3, 3, 3, 3, 3, 3]
8_1 [0, 0, 0, 0, 0, 0, 0, 0]
8_-1 [0, 0, 0, 1, 0, 0, 0, 0]
No own right lane [3, 3, 3, 3, 3, 3, 3, 3]
right_inner_lane [3, 3, 3, 3, 3, 3, 3, 3]
right_outer_lane [3, 3, 3, 3, 3, 3, 3, 3]
left_outer_lane [3, 3, 3, 3, 3, 3, 3, 3]
left_inner_lane [3, 3, 3, 3, 3, 3, 3, 3]
No opposing direction [3, 3, 3, 3, 3, 3, 3, 3]
8_1 [0, 0, 0, 0, 0, 0, 0, 0]
8_-1 [0, 0, 0, 1, 0, 0, 0, 0]
No own right lane [3, 3, 3, 3, 3, 3, 3, 3]
right_inner_lane [3, 3, 3, 3, 3, 3, 3, 3]
right_outer_lane [3, 3, 3, 3, 3, 3, 3, 3]
left_outer_lane [3, 3, 3, 3, 3, 3, 3, 3]
left_inner_lane [3, 3, 3, 3, 3, 3, 3, 3]
No opposing direction [3, 3, 3, 3, 3, 3, 3, 3]
8_1 [0, 0, 0, 0, 0, 0, 0, 0]
8_-1 [0, 0, 0, 1, 0, 0, 0, 0]
No own right lane [3, 3, 3, 3, 3, 3, 3, 3]
right_inner_lane [3, 3, 3, 3, 3, 3, 3, 3]
right_outer_lane [3, 3, 3, 3, 3, 3, 3, 3]
Junction ahead:
1 [3, 3, 3, 0, 0, 3, 3, 3]
2 [3, 3, 3, 

KeyboardInterrupt: 